In [1]:
import numpy as np
import pandas as pd
import numpy.ma as ma
from scipy.optimize import curve_fit
import numpy.linalg as LA
import math
import struct

#modulef = '/home/ubuntu/source/diff-classifier/diff_classifier/'
modulef = 'C:/Users/koolk/Desktop/diff-classifier/diff_classifier/'
import sys
sys.path.insert(0, modulef)

from utils import csv_to_pd
from msd import nth_diff, msd_calc, all_msds

In [ ]:
frames = 2000
drift = 0
d = {'Frame': np.linspace(1, frames, frames),
     'Track_ID': np.ones(frames),
             'X': np.random.rand(frames)+drift*np.linspace(0, 1, frames),
             'Y': np.random.rand(frames)+drift*np.linspace(0, 1, frames)}
df = pd.DataFrame(data=d)
df = all_msds(df)
#df

In [ ]:
def calculate_features(df):

    # Skeleton of Trajectory features metadata table.
    # Builds entry for each unique Track ID.
    die = {'Track_ID': df.Track_ID.unique(),
          'alpha': df.Track_ID.unique(),
          'D_fit': df.Track_ID.unique(),
          'kurtosis': df.Track_ID.unique(),
          'asymmetry1': df.Track_ID.unique(),
          'asymmetry2': df.Track_ID.unique(),
          'asymmetry3': df.Track_ID.unique(),
          'AR': df.Track_ID.unique(),
          'elongation': df.Track_ID.unique(),
          'boundedness': df.Track_ID.unique(),
          'fractal_dim': df.Track_ID.unique(),
          'trappedness': df.Track_ID.unique(),
          'efficiency': df.Track_ID.unique(),
          'straightness': df.Track_ID.unique(),
          'MSD_ratio': df.Track_ID.unique()}
    di = pd.DataFrame(data=die)

    trackids = df.Track_ID.unique()
    partcount = trackids.shape[0]


    for particle in range(0, partcount):
        single_track = df.loc[df['Track_ID'] == trackids[particle]].sort_values(['Track_ID', 'Frame'],
                                                                                 ascending=[1, 1]).reset_index(drop=True)
        di['alpha'], di['D_fit'] = alpha_calc(single_track)
        di['kurtosis'] = kurtosis(single_track)
        l1, l2, di['asymmetry1'], di['asymmetry2'], di['asymmetry3'] = asymmetry(single_track)
        di['AR'], di['elongation'] = aspectratio(single_track)
        di['boundedness'], di['fractal_dim'], di['trappedness'] = boundedness(single_track)
        di['efficiency'], di['straightness'] = efficiency(single_track)
        di['MSD_ratio'] = msd_ratio(single_track, 2, single_track['Frame'][single_track.shape[0]-2])
    
    return di

In [143]:
frames = 10
d = {'Frame': np.linspace(1, frames, frames),
             'X': np.linspace(1, frames, frames)+5,
             'Y': np.linspace(1, frames, frames)+3}
df = pd.DataFrame(data=d)
df['MSDs'], df['Gauss'] = msd_calc(df)

In [145]:
frames = 10
d = {'Frame': np.linspace(1, frames, frames),
             'X': np.sin(np.linspace(1, frames, frames))+3,
             'Y': np.cos(np.linspace(1, frames, frames))+3}
df = pd.DataFrame(data=d)
df['MSDs'], df['Gauss'] = msd_calc(df)

In [146]:
boundedness(df)

(0.96037058689895005, 2.9989749477908401, 0.03576118370932313)

In [124]:
import matplotlib.pyplot as plt
plt.plot(df['X'], df['Y'])
plt.show()

In [141]:
df['Frame'][df['Frame'].shape[0]-1]

10.0

In [142]:
def boundedness(track, framerate = 1):

    df = track
    length = df.shape[0]
    distance = np.zeros((length, length))

    for frame in range(0, length-1):
        distance[frame, 0:length-frame-1] = (np.sqrt(nth_diff(df['X'], frame+1)**2 + nth_diff(df['Y'], frame+1)**2).values)
    
    L = np.sum((np.sqrt(nth_diff(df['X'], 1)**2 + nth_diff(df['Y'], 1)**2).values))
    r = np.max(distance)/2
    f = df['Frame'][df['Frame'].shape[0]-1]*framerate
    D = df['MSDs'][length-1]/(4*f)
    
    B = D*f/(r**2)
    Df = np.log(length-1)/np.log((length-1)*2*r/L)
    pf = 1 - np.exp(0.2048 - 0.25117*(D*f/(r**2)))

    return B, Df, pf

In [ ]:
def efficiency(track):

    df = track
    length = df.shape[0]
    num = (nth_diff(df['X'], length-1)**2 + nth_diff(df['Y'], length-1)**2)[0]
    num2 = np.sqrt(num)
    
    den = np.sum(nth_diff(df['X'], 1)**2 + nth_diff(df['Y'], 1)**2)
    den2 = np.sum(np.sqrt(nth_diff(df['X'], 1)**2 + nth_diff(df['Y'], 1)**2))
    
    eff = num/den
    strait = num2/den2
    return eff, strait

In [ ]:
def msd_ratio(track, n1=3, n2=100):

    df = track
    assert n1 < n2, "n1 must be less than n2"
    ratio = (df['MSDs'][n1]/df['MSDs'][n2]) - (df['Frame'][n1]/df['Frame'][n2])
    return ratio

In [ ]:
msd_ratio(df, 1, 100)